In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/prediction-of-tourist-arrivals/submission.csv
/kaggle/input/prediction-of-tourist-arrivals/train_df.csv
/kaggle/input/prediction-of-tourist-arrivals/test_df.csv


## Import

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings 
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

In [3]:
train = pd.read_csv("/kaggle/input/prediction-of-tourist-arrivals/train_df.csv")
train.head()

,id,date,tourist_area,spot_facility,tourist_arrivals,area,city,type,category,tourism_index,info,event,weather_index
0,0,2018/8/1,1,A,454,A,A,A,13,1750.0,A,A,40.05
1,1,2018/8/1,1,B,823,A,A,A,13,1750.0,A,A,40.05
2,2,2018/8/1,1,C,149,A,A,A,13,1750.0,A,A,40.05
3,3,2018/8/1,1,D,157,A,A,A,13,1750.0,A,A,40.05
4,4,2018/8/1,1,E,14,A,A,A,13,1750.0,A,A,40.05


In [4]:
test = pd.read_csv("/kaggle/input/prediction-of-tourist-arrivals/test_df.csv")
test.head()

,id,date,tourist_area,spot_facility,area,city,type,category,tourism_index,info,event,weather_index
0,145584,2019/7/1,1,A,A,A,A,13,1362,A,A,NaN
1,145585,2019/7/1,1,B,A,A,A,13,1362,A,A,NaN
2,145586,2019/7/1,1,C,A,A,A,13,1362,A,A,NaN
3,145587,2019/7/1,1,D,A,A,A,13,1362,A,A,NaN
4,145588,2019/7/1,1,E,A,A,A,13,1362,A,A,NaN


In [5]:
sub = pd.read_csv("/kaggle/input/prediction-of-tourist-arrivals/submission.csv")
sub.head()

,id,tourist_arrivals
0,145584,0
1,145585,0
2,145586,0
3,145587,0
4,145588,0


## Feature Engineering

In [6]:
#日付への型変換
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

In [7]:
#曜日の取得
train['day_of_week'] = train['date'].dt.weekday
test['day_of_week'] = test['date'].dt.weekday

#年月日の取得
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day

train['year'] = train['year'].astype('int64')
train['month'] = train['month'].astype('int64')
train['day'] = train['day'].astype('int64')

test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day

test['year'] = test['year'].astype('int64')
test['month'] = test['month'].astype('int64')
test['day'] = test['day'].astype('int64')

In [8]:
# 祝日の取得
!pip install jpholiday
import jpholiday

In [9]:
holiday = pd.Series(pd.date_range(start='2018-08-01',end='2019-07-31'))

holidays = pd.DataFrame(
    {
      'holiday_name':holiday.apply(jpholiday.is_holiday_name),
      'date':holiday.values
    }
)

holidays = holidays[holidays['holiday_name'].notna()]

In [10]:
holidays['holiday'] = '1'
holidays['holiday'] = holidays['holiday'].astype('int64')
holidays = holidays.drop(['holiday_name'], axis=1)

In [11]:
train = pd.merge(train, holidays, on='date', how='left')
train['holiday'] = train['holiday'].fillna(0)
train.loc[train['day_of_week'] == 5, 'holiday'] = 1
train.loc[train['day_of_week'] == 6, 'holiday'] = 1

In [12]:
test = pd.merge(test, holidays, on='date', how='left')
test['holiday'] = test['holiday'].fillna(0)
test.loc[train['day_of_week'] == 5, 'holiday'] = 1
test.loc[train['day_of_week'] == 6, 'holiday'] = 1

In [13]:
def DATE(df):
    df['weekday']=(df['day_of_week']<5)
    df['Monday']=(df['day_of_week']==0)
    df['Tuesday']=(df['day_of_week']==1)
    df['Wednesday']=(df['day_of_week']==2)
    df['Thursday']=(df['day_of_week']==3)
    df['Friday']=(df['day_of_week']==4)
    df['Saturday']=(df['day_of_week']==5)
    df['Sunday']=(df['day_of_week']==6)
    
    df['Spring']=(df['month']>=3)&(df['month']<=5)
    df['Summer']=(df['month']>=6)&(df['month']<=8)
    df['Fall']=(df['month']>=9)&(df['month']<=11)
    df['Winter']=(df['month']>=12)|(df['month']<=2)
    
    return df

train = DATE(train)
test = DATE(test)

In [14]:
train

,id,date,tourist_area,spot_facility,tourist_arrivals,area,city,type,category,tourism_index,info,event,weather_index,day_of_week,year,month,day,holiday,weekday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Spring,Summer,Fall,Winter
0,0,2018-08-01,1,A,454,A,A,A,13,1750.0,A,A,40.05,2,2018,8,1,0.0,True,False,False,True,False,False,False,False,False,True,False,False
1,1,2018-08-01,1,B,823,A,A,A,13,1750.0,A,A,40.05,2,2018,8,1,0.0,True,False,False,True,False,False,False,False,False,True,False,False
2,2,2018-08-01,1,C,149,A,A,A,13,1750.0,A,A,40.05,2,2018,8,1,0.0,True,False,False,True,False,False,False,False,False,True,False,False
3,3,2018-08-01,1,D,157,A,A,A,13,1750.0,A,A,40.05,2,2018,8,1,0.0,True,False,False,True,False,False,False,False,False,True,False,False
4,4,2018-08-01,1,E,14,A,A,A,13,1750.0,A,A,40.05,2,2018,8,1,0.0,True,False,False,True,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132187,132187,2019-05-31,9,D,149,A,A,C,6,1998.0,A,A,48.29,4,2019,5,31,0.0,True,False,False,False,False,True,False,False,True,False,False,False
132188,132188,2019-05-31,9,E,56,A,A,C,6,1998.0,A,A,48.29,4,2019,5,31,0.0,True,False,False,False,False,True,False,False,True,False,False,False
132189,132189,2019-05-31,9,F,353,A,A,C,6,1998.0,A,A,48.29,4,2019,5,31,0.0,True,False,False,False,False,True,False,False,True,False,False,False
132190,132190,2019-05-31,9,G,35,A,A,C,6,1998.0,A,A,48.29,4,2019,5,31,0.0,True,False,False,False,False,True,False,False,True,False,False,False


In [15]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)


In [16]:
test['info_C']=False
test['info_D']=False
test['event_C']=False
test['event_D']=False
test['event_E']=False
test['event_F']=False

#特徴量作成
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import OneHotEncoder

cat_cols = ['spot_facility','area','city','type','info','event']

for c in cat_cols:
    le = LabelEncoder()
    le.fit(train[c])
    train[c] = le.transform(train[c])
    test[c] = le.transform(test[c])
    

In [17]:
test

,id,date,tourist_area,category,tourism_index,weather_index,day_of_week,year,month,day,holiday,weekday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Spring,Summer,Fall,Winter,spot_facility_A,spot_facility_B,spot_facility_C,spot_facility_D,spot_facility_E,spot_facility_F,spot_facility_G,spot_facility_H,area_A,area_B,area_C,area_D,area_E,area_F,area_G,area_H,area_I,area_J,area_K,area_L,area_M,area_N,area_O,area_P,area_Q,area_R,area_S,area_T,area_U,area_V,city_A,city_B,city_C,city_D,city_E,city_F,city_G,city_H,city_I,city_J,city_K,city_L,city_M,city_N,city_O,city_P,type_A,type_B,type_C,type_D,type_E,info_A,info_B,event_A,event_B,event_G,info_C,info_D,event_C,event_D,event_E,event_F
0,145584,2019-07-01,1,13,1362,NaN,0,2019,7,1,0.0,True,True,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False
1,145585,2019-07-01,1,13,1362,NaN,0,2019,7,1,0.0,True,True,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False
2,145586,2019-07-01,1,13,1362,NaN,0,2019,7,1,0.0,True,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False
3,145587,2019-07-01,1,13,1362,NaN,0,2019,7,1,0.0,True,True,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False
4,145588,2019-07-01,1,13,1362,NaN,0,2019,7,1,0.0,True,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13387,158971,2019-07-31,9,6,2040,50.21,2,2019,7,31,0.0,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,True,False,False,False,False,False,False,False,False
13388,158972,2019-07-31,9,6,2040,50.21,2,2019,7,

In [18]:
train_x = train.drop(['id','date','tourist_arrivals','weather_index'], axis=1)
train_y = train['tourist_arrivals']
test_x = test.drop(['id','date','weather_index'], axis=1)

Best parameters found:  {'reg_lambda': 0.5, 'reg_alpha': 0.5, 'num_leaves': 30, 'n_estimators': 3000, 'max_depth': 5, 'learning_rate': 0.015}

Best parameters found:  {'reg_lambda': 0.5, 'reg_alpha': 0.1, 'num_leaves': 30, 'n_estimators': 5000, 'max_depth': 7, 'learning_rate': 0.01}

Best parameters found:  {'reg_lambda': 0.3, 'reg_alpha': 0.5, 'num_leaves': 70, 'n_estimators': 5000, 'max_depth': 6, 'learning_rate': 0.015}

Best parameters found:  {'reg_lambda': 0.3, 'reg_alpha': 0.3, 'num_leaves': 70, 'n_estimators': 7000, 'max_depth': 7, 'learning_rate': 0.005}

Best parameters found:  {'reg_lambda': 0.5, 'reg_alpha': 0.1, 'num_leaves': 30, 'n_estimators': 6000, 'max_depth': 7, 'learning_rate': 0.02}

In [19]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV


params = {
    "objective" : "regression",
#     "objective" : "reg:squarederror",
    "metric" : "rmse",
#     "early_stopping_rounds":10,
    "random_state" : 2525
}

param_grid = {
    'n_estimators': [3000, 5000, 6000, 7000, 8000],  # ツリーの数
    'learning_rate': [0.005, 0.01, 0.015, 0.02],  # 学習率
    'max_depth': [5, 6, 7, 8],  # ツリーの最大深度
    'num_leaves': [30, 50, 60, 70, 80],  # 各ツリーの葉の最大数
    'reg_alpha': [0.1, 0.3, 0.5, 1.0],  # L1 正則化の強さ
    'reg_lambda': [0.3, 0.4, 0.5, 1.0],  # L2 正則化の強さ
}

#TimeSeriesSplitの設定
TSS = TimeSeriesSplit(n_splits = 4)

# ランダムサーチの設定
random_search = RandomizedSearchCV(
    estimator=lgb.LGBMRegressor(**params),
#     estimator=xgb.XGBRegressor(**params),
    param_distributions=param_grid,
    scoring='neg_mean_squared_error',  # モデルの評価指標（ここでは平均二乗誤差）を指定
    cv=TSS  # 時系列交差検証法による分割を指定
)

# モデルの学習とチューニング
random_search.fit(train_x, train_y)

# ベストなパラメータとスコアの表示
print("Best parameters found: ", random_search.best_params_)
print("Best negative mean squared error found: ", random_search.best_score_)

# ベストなパラメータでのモデル構築と学習
best_params = random_search.best_params_

# 交差検証の実行
for tr_idx, va_idx in TSS.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    model = lgb.LGBMRegressor(**params, **best_params)
#     model = xgb.XGBRegressor(**params, **best_params)
    model.fit(tr_x, tr_y)

    # 検証データでの予測
    y_pred = model.predict(va_x)

    # 平均二乗誤差の計算
    mse = mean_squared_error(va_y, y_pred)
    rmse = np.sqrt(mse)

    # モデルの評価
    print("RMSE: ", rmse)

Best parameters found:  {'reg_lambda': 0.5, 'reg_alpha': 0.1, 'num_leaves': 30, 'n_estimators': 8000, 'max_depth': 7, 'learning_rate': 0.015}
Best negative mean squared error found:  -12220.229579129977
RMSE:  99.200400692692
RMSE:  148.93504115683038
RMSE:  87.48438283183803
RMSE:  95.94287412348507


In [20]:
model = lgb.LGBMRegressor(**params, **best_params)
model.fit(train_x, train_y)

y_pred = model.predict(test_x)

In [21]:
sub.iloc[:, -1] = y_pred
sub

,id,tourist_arrivals
0,145584,500.257217
1,145585,945.154325
2,145586,210.026094
3,145587,164.074026
4,145588,34.498697
...,...,...
13387,158971,298.412210
13388,158972,60.540079
13389,158973,460.201292
13390,158974,53.823088


In [22]:
sub.to_csv("submission.csv",index=None)
sub.head()

,id,tourist_arrivals
0,145584,500.257217
1,145585,945.154325
2,145586,210.026094
3,145587,164.074026
4,145588,34.498697
